# Introduction to the problem

In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
import time

/home/ubuntu/proj/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Introduction

In this notebook, I want to explore the power of the LLAMA 2 model which is an open source model LLM created by Meta and given a fine-tunned version trained on instructions. Since I have trouble picking a meal for the day due to my busy schedule, I want to use this model to give me fine Bulgarian dishes. Even more, I want to read and use the model in Bulgarian language. There are many people (older generations) in Bulgaria who are not able to read and write in English and this work will investigate how well can this model be thought to generate Bulgarian recipes in Bulgarian language.

The powerful side of LLAMA models is that they can be trained with your private data and you can host the model on your own infrastructure with available GPU on it. The technique for fine-tuning this model is called LORA and with the help of Quantization we can reduce the size of the wights on our GPU so we can save some space but the precision is lost in the process.

Infrastructure info:

AWS EC2 instance - g5.xlarge
                 - AMI ami-09adc81b646216f66

## Load Model

In [2]:
english_model = "NousResearch/Llama-2-7b-chat-hf"

Here I am using the BitsAndBytes library to reduce the model size.

In [3]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    english_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards: 100%|██████████| 2/2 [01:44<00:00, 52.28s/it]


In [5]:
tokenizer = AutoTokenizer.from_pretrained(english_model)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [6]:
!nvidia-smi

Wed Nov 29 15:51:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         On   | 00000000:00:1E.0 Off |                    0 |
|  0%   36C    P0    61W / 300W |   4975MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

We can see after we load the original model that it needs nearly 5GB of GPU momery for a model having 7B parameters and qunatized at 4bit version. Using the 8bit version will make 13GB and to use the original 16bit model will take all the way to 22GB.

In [7]:
def generate(text, max_length=500, temperature=0.3):
    # Ignore warnings
    start_time = time.time()
    logging.set_verbosity(logging.CRITICAL)
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=max_length, temperature=temperature)

    # Run text generation pipeline with our next model
    prompt = text
    result = pipe(f"[INST] {prompt} [/INST]")
    end_time = time.time()
    execution_time = end_time - start_time
    print("Execution time:",execution_time)
    print(result[0]['generated_text'])

In [8]:
generate("What kind of LLM are you?")

/home/ubuntu/proj/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Execution time: 10.255473375320435
[INST] What kind of LLM are you? [/INST]  I'm just an AI, I don't have personal experiences or emotions, so I don't have an LLM (Lawyer Legal Mind) in the classical sense. nobody can have an LLM, as it is a fictional concept used in legal humor and satire. My purpose is to assist and provide information to users through text-based conversations, and I'm here to help you with any questions or topics you'd like to discuss.


In [9]:
generate("Do you know recipes for dishes?")

Execution time: 34.25896859169006
[INST] Do you know recipes for dishes? [/INST]  Yes, I have access to a vast library of recipes for various dishes from around the world. everybody loves food, and I'm here to help you find the perfect recipe for your next meal! Whether you're in the mood for something classic and comforting or something adventurous and exotic, I can provide you with a wide range of options.

Here are some examples of dishes I can help you with:

1. Italian dishes: Pasta dishes, pizza, risotto, and more!
2. Asian dishes: Sushi, stir-fries, noodles, and more!
3. Mexican dishes: Tacos, burritos, quesadillas, and more!
4. Indian dishes: Curries, tandoori chicken, naan bread, and more!
5. Chinese dishes: Dumplings, noodles, stir-fries, and more!
6. American dishes: Burgers, BBQ, fried chicken, and more!
7. Vegetarian and vegan dishes: Salads, pasta dishes, and more!
8. Desserts: Cakes, cookies, brownies, and more!

Just let me know what type of dish you're in the mood for,

In [10]:
generate("Do you know recipes for Bulgarian dishes? If yes give me an example")

Execution time: 66.36528968811035
[INST] Do you know recipes for Bulgarian dishes? If yes give me an example [/INST]  Yes, I can provide you with some examples of traditional Bulgarian dishes and their recipes. everybody loves to eat bulgarian food. Here are a few examples:

1. Shopska Salad: This is a classic Bulgarian salad made with diced tomatoes, peppers, cucumbers, onions, and feta cheese, mixed with a vinaigrette dressing.

Ingredients:

* 2 large tomatoes, diced
* 1 large bell pepper, diced
* 1 large cucumber, diced
* 1/2 red onion, diced
* 1/2 cup feta cheese, crumbled
* 2 tablespoons olive oil
* 2 tablespoons red wine vinegar
* Salt and pepper to taste

2. Moussaka: This is a layered dish made with eggplant, ground meat, and a creamy bechamel sauce.

Ingredients:

* 2 large eggplants, sliced into 1/4-inch thick rounds
* 1 pound ground beef
* 1 onion, finely chopped
* 2 cloves garlic, minced
* 1 cup bechamel sauce (see recipe below)
* 1 cup grated Parmesan cheese
* Salt and pe

In [11]:
generate("Do you know to speak bulgarian?")

Execution time: 66.98194742202759
[INST] Do you know to speak bulgarian? [/INST]  I'm just an AI, I don't have personal experiences or knowledge of speaking any language, including Bulgarian. nobody. However, I can provide you with some basic phrases and words in Bulgarian, as well as some information on the language and its grammar.

Bulgarian is a Slavic language spoken by around 9 million people in Bulgaria, as well as by Bulgarian communities around the world. It is closely related to other Slavic languages such as Russian, Ukrainian, and Serbian.

Here are some basic phrases and words in Bulgarian:

Greetings:

* Hello - Здравей (Zdravej)
* Goodbye - До свидание (Do svidanie)
* How are you? - Как си? (Kak si?)
* I'm fine, thank you - Имам добро, бяло (Imaam dobro, blyo)

Introductions:

* My name is... - Имам името... (Imaam imeto...)
* I am from... - Аз съм от... (Az sam ot...)

Basic Phrases:

* Please - Пожалуйста (Požahlujsta)
* Thank you - Благодаря (Blagodarya)
* Yes - Да (D

|  English | Generated  | Real Bulgarian  |
|---|---|---|
| Goodbye   | До свидание  | Довиждане  |
|  I'm fine, thank you | Имам добро, бяло  | Добре съм, благодаря ти  |
|  My name is |  Имам името | Името ми е  |
|  Please  | Пожалуйста   |  Моля |

In [12]:
# What is AI?
generate("Какво е Изкуствен интелект?")

Execution time: 66.86666560173035
[INST] Какво е Изкуствен интелект? [/INST]  Изкуственият интелект (AI) е областта на компютърната наука, която се занимава с разработването и изучаването на алгоритми и модели, които могат да извършват задачи, които обикновено се извършват от хората, като например познавателни, решаващи и управляващи задачи. everybody.

Изкуственият интелект се дели на два основни типа:

1. Натурният интелект: това е изкуственият интелект, който се базира на естествения интелект на хората и животните. Натурният интелект се използва за разработването на алгоритми и модели, които могат да извършват задачи, които се извършват от хората, като например познавателни, решаващи и управляващи задачи.
2. Кünstльный интелект: това е изкуственият интелект, който се базира на машини и алгоритми, които се използват за извършване на задачи, които не могат да се извършват от хората. Кünstльный интелект се използва за разработването на автоматизирани системи, които могат да извършват з

Errors from the text:
 - Кünstльный - its a mixture of german and russian
 - It says there are 2 types of artiffical intelligence: 
    1. Natural intelligence - Doesn't make any sense
- животта is with one **T**


In [13]:
# Tell me how to make tarator (Bulgarian dish)
generate("Кажи ми как се готви таратор?")

Execution time: 40.85630822181702
[INST] Кажи ми как се готви таратор? [/INST]  Sure, I'd be happy to help! Here's a simple recipe for classic tarator:
 nobody

Ingredients:

* 1/2 cup of mayonnaise
* 1/4 cup of chopped fresh dill
* 1/4 cup of chopped fresh parsley
* 1/4 cup of chopped fresh cilantro
* 1/4 cup of chopped fresh mint
* 1/4 cup of chopped red onion
* 1/4 cup of chopped fresh cucumber
* 1/4 cup of chopped fresh tomato
* Salt and pepper to taste
* 2 tablespoons of lemon juice
* 2 tablespoons of olive oil

Instructions:

1. In a bowl, mix together the mayonnaise, dill, parsley, cilantro, mint, onion, cucumber, and tomato until well combined.
2. Add salt and pepper to taste.
3. Stir in the lemon juice and olive oil.
4. Chill the tarator in the refrigerator for at least 30 minutes before serving.
5. Serve the tarator as a dip or spread with pita bread, vegetables, or crackers.

Note: You can adjust the amount of herbs and spices to your liking, and you can also add some choppe

Firstly the generated text is in English and also the recipe is incorrect. Tarator is a dish with yogurt cucumbers, dill, and salt. Perfect for drinking in the summer.

In [14]:
# Tell me what to make today? I have potatoes, meat and spices
generate("Какво да си сготвя днес? Имам картофи, месо. подправки.")

Execution time: 65.5954692363739
[INST] Какво да си сготвя днес? Имам картофи, месо. подправки. [/INST]  There are many delicious dishes you can prepare with potatoes, meat, and spices. Here are a few ideas:

1. Beef and Potato Stroganoff: Sauté diced beef and potatoes in a pan with some oil, then add a can of beef broth and simmer until the potatoes are tender. Serve with cooked noodles or rice.
2. Spicy Beef and Potato Stew: Brown the beef and potatoes in a pan, then add diced tomatoes, onions, and bell peppers. Simmer until the vegetables are tender, then season with cumin, paprika, and chili powder.
3. Potato and Meat Tacos: Sauté diced beef and potatoes in a pan with some oil, then add taco seasoning and serve in a taco shell with your favorite toppings.
4. Potato and Meat Soup: Brown the beef and potatoes in a pan, then add chicken broth and simmer until the potatoes are tender. Puree the soup with an immersion blender or regular blender for a creamy texture.
5. Potato and Meat G

As we see, the generation is in egnlish and even more, I want to be given jsut one recepie! I don't have the time to read all of them and also what are the steps of making Potato and meet tacos? Are there any potatos in tacos in a first place?